In [66]:
import pandas as pd
import numpy as np
from scipy import sparse as sps
import nltk
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
from scipy import sparse
from sklearn.preprocessing import normalize
lemmatizer = WordNetLemmatizer() 
words = set(nltk.corpus.words.words())
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
saved = open("pickle/game_reviews.pickle","rb")
game_reviews = pickle.load(saved)
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [6]:
reviews = game_reviews[['item_id', 'user_id','review']]
reviews = reviews.rename(columns={'user_id': 'user', 'item_id': 'item'})
reviews.shape

(58430, 3)

In [7]:
user_game = open("pickle/user_games.pickle","rb")
user_games = pickle.load(user_game)
user_games.head()

,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [8]:
user_games_list = user_games.rename(columns={'user_id': 'user', 'item_id': 'item'})
user_games_list = user_games_list[['item', 'user']]

user_games_list.head()

,item,user
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479


In [9]:
def process(content):
        
        processed = tokenizer.tokenize(content)

        processed = [token.lower() for token in processed]

        processed = [token for token in processed if token not in stopwords.words('english')]
        
        #processed = [token for token in processed if token in words or not token.isalnum()]
        
        processed = [ps.stem(token) for token in processed]
        
        return processed

In [93]:
class LemmaTokenizer(object):
        
    def __call__(self, content):

        processed = tokenizer.tokenize(content)
        processed = [token.lower() for token in processed]
        processed = [token for token in processed if token not in stopwords.words('english')]
        processed = [ps.stem(token) for token in processed]

        return processed
            
            #return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [10]:
item_data = pd.DataFrame({'review': reviews.groupby(['item']).review.apply(lambda x:' '.join(x))})
item_data.reset_index(inplace=True)
item_data['processed_reviews'] = item_data['review'].apply(lambda row: process(row))

In [94]:
vect = CountVectorizer(tokenizer=LemmaTokenizer())

In [99]:
BOS = item_data['review'].tolist()

In [101]:
X = vect.fit_transform(BOS)
X

In [152]:
X.todense()

MemoryError: 

In [114]:
sum_words = X.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
#words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:15]

[('cool', 1199),
 ('game', 66857),
 ('wallbang', 2),
 ('simul', 1063),
 ('1', 2531),
 ('onlin', 931),
 ('action', 941),
 ('awesom', 3660),
 ('better', 3088),
 ('team', 1688),
 ('fortress', 422),
 ('mutiplit', 1),
 ('player', 3389),
 ('best', 5842),
 ('fp', 1776)]

In [105]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer1=TfidfTransformer()

a = tfidf_transformer1.fit_transform(X)

In [67]:
#vect = CountVectorizer(tokenizer=LemmaTokenizer())
item_data['count'] = item_data['review'].apply(lambda row: CountVectorizer(tokenizer=process(row)).)
item_data['count'].fit_transform()

In [12]:
#item_data['bow'] = item_data['processed_reviews'].apply(lambda row: ' '.join(row))
#bow = item_data['bow'].tolist()
#len(bow)

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

In [14]:
bow = item_data['processed_reviews'].tolist()
len(bow)

3682

In [31]:
def tuple_to_dict(row):
    dc = dict((x, y) for x, y in row)
    return dc

In [404]:
new.shape

(3682, 70964)

In [413]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer1=TfidfTransformer()

a = tfidf_transformer1.fit_transform(new)

In [418]:
a.todense().shape

(3682, 70964)

In [320]:
corpus_list = []
for item in item_data['processed_reviews']:
    corpus_list.append(' '.join(item))
    #corpus_list.append(item)
tfidf_matrix = TfidfVectorizer().fit_transform(corpus_list)
tfidf_matrix[1]

<1x70710 sparse matrix of type '<class 'numpy.float64'>'
	with 190 stored elements in Compressed Sparse Row format>

In [417]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(bow) 
corpus = [dictionary.doc2bow(text) for text in bow]

In [334]:
from gensim.sklearn_api import TfIdfTransformer
model = TfIdfTransformer(dictionary)
tfidf_corpus = model.fit_transform(corpus)

In [338]:
type(tfidf_corpus[0][0])

tuple

In [19]:
item_data['doc2bow'] = corpus

In [20]:
item_data.head()

,item,review,processed_reviews,doc2bow
0,10,Cool game Wallbang simulator this game is the ...,"[cool, game, wallbang, simul, game, 1, onlin, ...","[(0, 4), (1, 7), (2, 1), (3, 1), (4, 1), (5, 1..."
1,20,No Bread. Team Fortress tutkusunun başlangıcı;...,"[bread, team, fortress, tutkusunun, başlangıcı...","[(0, 1), (1, 2), (5, 3), (20, 1), (24, 1), (39..."
2,30,An excellent war shooter with a great online c...,"[excel, war, shooter, great, onlin, commun, 5,...","[(0, 1), (1, 1), (5, 1), (65, 1), (105, 2), (1..."
3,40,Best playerbase I have ever met. Everybody is ...,"[best, playerbas, ever, met, everybodi, nice, ...","[(71, 1), (82, 1), (185, 1), (225, 1), (241, 1..."
4,50,You want a decent Half-Life 1 game that's chea...,"[want, decent, half, life, 1, game, cheap, get...","[(0, 1), (72, 1), (91, 1), (119, 1), (225, 3),..."


In [21]:
len(corpus[0])

571

In [22]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of articles: %d' % len(corpus))

Number of unique tokens: 70964
Number of articles: 3682


In [32]:
dictvectorizer = DictVectorizer(sparse=True)
dict_val = item_data['doc2bow'].apply(lambda row: tuple_to_dict(row))
count_vec = dictvectorizer.fit_transform(dict_val)
count_vec
#tf_idf_mat = tfidf_transformer.fit_transform(count_vec)

<3682x70964 sparse matrix of type '<class 'numpy.float64'>'
	with 579792 stored elements in Compressed Sparse Row format>

In [38]:
lda = LatentDirichletAllocation(n_components=20, random_state=0)
a = lda.fit_transform(count_vec)
#lda.transform(bow)

In [63]:
count_vec.shape

(3682, 70964)

In [57]:
sA = sparse.csr_matrix(a)

In [58]:
sA.toarray()

array([[5.02512563e-05, 5.02512573e-05, 5.02512565e-05, ...,
        5.02512566e-05, 5.02512569e-05, 5.02512577e-05],
       [1.70648464e-04, 1.70648468e-04, 1.70648465e-04, ...,
        1.70648465e-04, 1.70648466e-04, 1.70648467e-04],
       [5.88235294e-04, 5.88235304e-04, 5.88235296e-04, ...,
        5.88235297e-04, 5.88235301e-04, 5.88235299e-04],
       ...,
       [1.25000000e-02, 1.25000003e-02, 1.25000000e-02, ...,
        1.25000000e-02, 1.25000001e-02, 1.25000002e-02],
       [1.04166667e-03, 1.04166676e-03, 1.04166667e-03, ...,
        1.04166668e-03, 1.04166667e-03, 1.04166668e-03],
       [4.54545455e-03, 4.54545468e-03, 4.54545455e-03, ...,
        4.54545455e-03, 4.54545460e-03, 4.54545467e-03]])

In [115]:
import gensim
NUM_TOPICS = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, per_word_topics=True)
#ldamodel.save('model5.gensim')

In [151]:
lda_model.show_topic(topicid=0, topn=10)

[('isaac', 0.011600827), ('nope', 0.009645291), ('awesom', 0.009447839), ('bind', 0.009280967), ('10', 0.008524305), ('starv', 0.006257442), ('end', 0.005372209), ('rebirth', 0.0051313103), ('stanley', 0.0045404444), ('ftl', 0.0043492103)]

In [133]:
lda_model.print_topics()

[(0, '0.012*"isaac" + 0.010*"nope" + 0.009*"awesom" + 0.009*"bind" + 0.009*"10" + 0.006*"starv" + 0.005*"end" + 0.005*"rebirth" + 0.005*"stanley" + 0.004*"ftl"'), (1, '0.026*"и" + 0.020*"в" + 0.017*"не" + 0.012*"что" + 0.012*"на" + 0.011*"это" + 0.009*"игра" + 0.009*"но" + 0.008*"я" + 0.008*"с"'), (2, '0.045*"game" + 0.015*"10" + 0.013*"play" + 0.008*"nyan" + 0.008*"like" + 0.007*"good" + 0.007*"get" + 0.006*"great" + 0.006*"fun" + 0.006*"de"'), (3, '0.084*"bad" + 0.019*"horror" + 0.018*"took" + 0.018*"dad" + 0.017*"shower" + 0.015*"game" + 0.013*"scari" + 0.010*"scare" + 0.009*"dosh" + 0.008*"10"'), (4, '0.002*"osx" + 0.001*"hoi" + 0.001*"แฟน" + 0.001*"recommmend" + 0.000*"beatbox" + 0.000*"switzerland" + 0.000*"armageddon" + 0.000*"goblin" + 0.000*"surivi" + 0.000*"guerilla"'), (5, '0.056*"game" + 0.018*"10" + 0.018*"play" + 0.012*"fun" + 0.012*"get" + 0.011*"good" + 0.009*"zombi" + 0.008*"2" + 0.008*"friend" + 0.008*"like"'), (6, '0.073*"shit" + 0.025*"hit" + 0.014*"arma" + 0.009*"v

In [116]:
def embedding(data_table, column_name, new_col):
    data_table[new_col] = data_table[column_name]
    #data_table[new_col] = data_table[new_col].apply(lambda row: lda_model.get_document_topics(row))
    data_table[new_col] = data_table[new_col].apply(lambda row: lda_model[row])
    return data_table

In [117]:
embedding(item_data, 'doc2bow', 'LDA_topic').head()

,item,review,processed_reviews,doc2bow,LDA_topic,count
0,10,Cool game Wallbang simulator this game is the ...,"[cool, game, wallbang, simul, game, 1, onlin, ...","[(0, 4), (1, 7), (2, 1), (3, 1), (4, 1), (5, 1...","([(2, 0.27237153), (5, 0.18286814), (9, 0.2706...","CountVectorizer(analyzer='word', binary=False,..."
1,20,No Bread. Team Fortress tutkusunun başlangıcı;...,"[bread, team, fortress, tutkusunun, başlangıcı...","[(0, 1), (1, 2), (5, 3), (20, 1), (24, 1), (39...","([(5, 0.15171626), (6, 0.095982514), (9, 0.348...","CountVectorizer(analyzer='word', binary=False,..."
2,30,An excellent war shooter with a great online c...,"[excel, war, shooter, great, onlin, commun, 5,...","[(0, 1), (1, 1), (5, 1), (65, 1), (105, 2), (1...","([(8, 0.012374228), (9, 0.15228565), (13, 0.11...","CountVectorizer(analyzer='word', binary=False,..."
3,40,Best playerbase I have ever met. Everybody is ...,"[best, playerbas, ever, met, everybodi, nice, ...","[(71, 1), (82, 1), (185, 1), (225, 1), (241, 1...","([(3, 0.060778096), (5, 0.89185345)], [(71, [5...","CountVectorizer(analyzer='word', binary=False,..."
4,50,You want a decent Half-Life 1 game that's chea...,"[want, decent, half, life, 1, game, cheap, get...","[(0, 1), (72, 1), (91, 1), (119, 1), (225, 3),...","([(14, 0.9586956)], [(0, [14]), (72, [14]), (9...","CountVectorizer(analyzer='word', binary=False,..."


In [144]:
item_data['processed_reviews'].loc[100]

['legendari', 'awesom', 'game', 'definit', 'worth', 'even', 'star', 'war', 'fan', 'still', 'lot', 'fun', 'game', '10', '10', 'probabl', 'one', 'difficult', 'game', 'ever', 'creat', 'love', 'everi', 'singl', 'aspect', 'worth', 'frustrat', '100']

In [143]:
item_data['LDA_topic'].loc[100]

([(13, 0.35756117), (14, 0.61140436)], [(1, [14, 13]), (2, [14, 13]), (60, [14, 13]), (183, [14, 13]), (185, [14, 13]), (223, [14]), (224, [14, 13]), (225, [14, 13]), (317, [14, 13]), (319, [14, 13]), (376, [14, 13]), (497, [14, 13]), (550, [13, 14]), (601, [14, 13]), (843, [14, 13]), (941, [14, 13]), (964, [14, 13]), (983, [14, 13]), (1061, [13, 14]), (1378, [14, 13]), (1464, [14, 13]), (1605, [14, 13]), (3090, [13, 14]), (6327, [13, 14])], [(1, [(13, 0.9342886), (14, 1.0657115)]), (2, [(13, 0.32101214), (14, 0.6789878)]), (60, [(13, 0.47876626), (14, 0.52123374)]), (183, [(13, 0.26483765), (14, 0.7351623)]), (185, [(13, 0.48363823), (14, 0.51636183)]), (223, [(14, 1.0)]), (224, [(13, 0.39163318), (14, 0.6083668)]), (225, [(13, 0.9135854), (14, 2.0864146)]), (317, [(13, 0.22128966), (14, 0.7787103)]), (319, [(13, 0.47521734), (14, 0.52478266)]), (376, [(13, 0.33065966), (14, 0.6693402)]), (497, [(13, 0.38584885), (14, 0.6141511)]), (550, [(13, 0.9653385), (14, 0.034661487)]), (601, [(

In [44]:
def tupletodic(row):
    dc = dict((x, y) for x, y in row)
    return dc

In [45]:
from sklearn.feature_extraction import DictVectorizer
dictvectorizer = DictVectorizer(sparse=True)

In [46]:
dict_val = item_data['LDA_topic'].apply(lambda row: tupletodic(row))
new = dictvectorizer.fit_transform(dict_val)

In [51]:
new.shape

(3682, 20)

In [64]:
new[0].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.0520336 ,
        0.        , 0.        , 0.        , 0.11329363, 0.        ,
        0.        , 0.        , 0.        , 0.57879072, 0.        ,
        0.        , 0.        , 0.25507805, 0.        , 0.        ]])

In [47]:
norm_mat = normalize(new, norm='l2', axis=1)
cosine_mat = norm_mat @ norm_mat.T
#cosine_mat = new @ new.T
cosine_mat = cosine_mat.toarray()

In [48]:
cosine_mat

array([[1.        , 0.36037342, 0.1756809 , ..., 0.41353664, 0.17573746,
        0.32675077],
       [0.36037342, 1.        , 0.96853868, ..., 0.02167847, 0.96885048,
        0.07651325],
       [0.1756809 , 0.96853868, 1.        , ..., 0.0167613 , 0.99967818,
        0.        ],
       ...,
       [0.41353664, 0.02167847, 0.0167613 , ..., 1.        , 0.01635175,
        0.02118149],
       [0.17573746, 0.96885048, 0.99967818, ..., 0.01635175, 1.        ,
        0.        ],
       [0.32675077, 0.07651325, 0.        , ..., 0.02118149, 0.        ,
        1.        ]])

In [500]:
def itemid_2_index():
        r_index = pd.Index(item_data.item.unique(), name='item')
        return r_index

In [501]:
def get_user_item(userID):
        user_item_ids = review_data.set_index('user')['item']
        user_item = user_item_ids.loc[userID]
        if isinstance(user_item, str):
            user_item = pd.Series(user_item).rename("item")
        temp_df = user_item.to_frame()
        temp_df = temp_df.reset_index()
        return temp_df

In [502]:
def score_reviews(item):
        try:
            item2index = itemid_2_index()
            idx = item2index.get_loc(item)
        except KeyError:
            return pd.Series(0, item2index, name='rev_sim')
        row = cosine_mat[idx, :].copy()
        row[idx] = 0
        item_sim = pd.Series(row, item2index, name='rev_sim')
        return item_sim

In [503]:
def predict_for_user(userID, itemList, ratings = None):
        user_item_ids = review_data.set_index('user')['item']
        if userID in user_item_ids.index:
            temp_df = get_user_item(userID)
            items = temp_df[:]['item']
            scores = items.apply(lambda x: score_reviews(x))
            present = items[items.isin(scores.columns)]
            scores.loc[:, present] = 0
            predList = scores.filter(items=itemList)
            final_score = predList.sum(axis=0)
            return final_score

In [504]:
user_games.head()

,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [505]:
review_data = reviews
predict_for_user('evcentric',[10,20,30,40,50,60])

item
10    2.259903
20    3.095146
30    2.041466
40    2.483646
50    4.041049
60    1.817818
dtype: float64

In [439]:
#gensim.matutils.cossim(item_data['LDA_topic'].loc[0], item_data['LDA_topic'].loc[1])

In [18]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(ldamodel.print_topics())
doc_lda = ldamodel[corpus]

[(0,
  '0.021*"path" + 0.019*"exil" + 0.013*"plane" + 0.011*"war" + 0.010*"diablo" '
  '+ 0.009*"star" + 0.008*"nope" + 0.006*"gear" + 0.006*"emili" + '
  '0.005*"metal"'),
 (1,
  '0.040*"de" + 0.034*"que" + 0.016*"la" + 0.015*"lo" + 0.013*"juego" + '
  '0.013*"game" + 0.012*"un" + 0.012*"el" + 0.012*"en" + 0.011*"es"'),
 (2,
  '0.016*"spoler" + 0.016*"spolier" + 0.015*"spoiler" + 0.004*"stealth" + '
  '0.002*"runescap" + 0.001*"bottleneck" + 0.001*"cs" + 0.001*"cpu" + '
  '0.001*"rs" + 0.001*"ninja"'),
 (3,
  '0.041*"dlc" + 0.019*"payday" + 0.017*"heist" + 0.015*"updat" + 0.012*"add" '
  '+ 0.012*"new" + 0.012*"overkil" + 0.011*"overpr" + 0.010*"2" + '
  '0.009*"microtransact"'),
 (4,
  '0.040*"ㅤ" + 0.035*"game" + 0.019*"mod" + 0.014*"garri" + 0.011*"fun" + '
  '0.010*"sandbox" + 0.009*"hey" + 0.008*"play" + 0.008*"10" + '
  '0.007*"gamemod"'),
 (5,
  '0.045*"game" + 0.013*"play" + 0.009*"like" + 0.009*"10" + 0.007*"get" + '
  '0.007*"one" + 0.007*"good" + 0.007*"great" + 0.006*"stori